In [1]:
# Note: this cell is hidden from the HTML output. Read more: https://nbsphinx.readthedocs.io/en/0.2.1/hidden-cells.html
import jupyter_black

jupyter_black.load()

# User Guide: Quick Start

Welcome to the Quickstart User Guide for `sec-parser`! This guide is designed to walk you through the fundamental steps needed to install and use the library for parsing SEC EDGAR HTML documents into semantic elements and trees. Whether you're a financial analyst, a data scientist, or someone interested in SEC filings, this guide provides examples and code snippets to help you get started.

This guide is interactive, allowing you to engage with the code and concepts as you learn. You can run and modify all the code examples shown here for yourself by cloning the repository and running the [user_guide.ipynb](https://github.com/alphanome-ai/sec-parser/blob/main/docs/source/notebooks/user_guide.ipynb) in a Jupyter notebook.

Alternatively, you can also run the notebook directly in your browser using Google Colab:

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/alphanome-ai/sec-parser/blob/main/docs/source/notebooks/user_guide.ipynb)
[![My Binder](https://mybinder.org/badge.svg)](https://mybinder.org/v2/gh/alphanome-ai/sec-parser/main?filepath=docs/source/notebooks/user_guide.ipynb)

Let's get started!